<a href="https://colab.research.google.com/github/retnuh-k/peptide-mutation-finder/blob/main/PeptideMutationFinder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Mutation to peptide sequence pipeline

This notebook automates the process of figuring out changes in protein sequences as a result of mutations found from genome sequencing. Written by Hunter King (https://github.com/retnuh-k/) under the guidance of Dr. Songtao Jia and Yimeng Fang as part of the Jia Lab (http://jia.biology.columbia.edu/).

Sample files available on github: https://github.com/retnuh-k/peptide-mutation-finder 


**Required input files:** 
*   mutation_comparison_tsv_file - file describing the differences in two vcf files outputted by vcftools --gzdiff function (see http://vcftools.sourceforge.net/man_latest.html)
*   geneome_tsv_file - a tsv file of the organism's feature coordinates in gff3 file format (can create by removing header starting with '#' from gff3 file and saving as tsv)
*   genome_fasta_file - a fasta file with chromosome headers written as ">[chromosome name]"

**Full pipeline steps**
1.   Obtain sequencing data of strain of interest and wild type
2.   Download reference genome fasta file
3.   Run freebayes variant detector (see https://github.com/freebayes/freebayes) on both strain and wild type .bam files against the reference genome to get a vcf (variant call file) for each
4.   Filter each vcf file by quality (recommended between 10 and 30) and (optionally) depth using vcffilter or equivalent (https://github.com/vcflib/vcflib)
5.   gzip filtered vcf files
6.   Compare vcf files using vcftools (http://vcftools.sourceforge.net/man_latest.html), run "vcftools --gzvcf test_strain.vcf.gz --gzdiff wild_time.vcf.gz --diff-site --out comparison_output_file". This step ensures that only mutations not shared with the wild type are analyzed.
7.   Rename comparison output file to have .tsv file extension
8.   Download genome fasta file and reformat per the above specifications
9.   Download genome coordinate gff3 file, remove header and save as .tsv file
10.  Upload required files to notebook, specify filenames in code block below, and run notebook 



In [1]:
#@title (Optional) Download sample files

!git clone https://github.com/retnuh-k/peptide-mutation-finder.git

Cloning into 'peptide-mutation-finder'...
remote: Enumerating objects: 38, done.
remote: Counting objects: 100% (38/38), done.
remote: Compressing objects: 100% (37/37), done.
remote: Total 38 (delta 18), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (38/38), done.


In [3]:
import numpy as np
import pandas as pd
import re
try:
  import Bio
except:
  !pip install Biopython
from Bio.Seq import Seq
from google.colab import files

#@title Input filenames, then hit Runtime -> Run All
mutation_comparison_tsv_file = "peptide-mutation-finder/TT_v_WT_DP10QU20.tsv" #@param {type:"string"}
geneome_tsv_file = "peptide-mutation-finder/Schizosaccharomyces_pombe_all_chromosomes.tsv" #@param {type:"string"}
genome_fasta_file = "peptide-mutation-finder/pombe_genome.fasta" #@param {type:"string"}
job_name = "mutation_results" #@param {type:"string"}
download_mutation_csv = True #@param {type:"boolean"}

# Load Dataframes
mut_df = pd.read_csv(mutation_comparison_tsv_file,sep='\t',header=0)
mask = mut_df['IN_FILE']=='1'
mut_df_1 = mut_df[mask]
genome_df = pd.read_csv(geneome_tsv_file,
                        sep='\t',
                        names=["chromosome","source","seq_type","start_pos","end_pos","score","strand","frame","id"])

exon_m = genome_df['id'].str.contains("exon")
exon_df = genome_df[exon_m]
for row in exon_df.index:
  if "RNA" in str(exon_df.loc[row,"id"]):
    exon_df.drop(row,inplace=True)

mut_df_1.insert(mut_df_1.shape[1],"gene_id",np.empty(mut_df_1.shape[0]))
mut_df_1.insert(mut_df_1.shape[1],"gene_start",np.empty(mut_df_1.shape[0]))
mut_df_1.insert(mut_df_1.shape[1],"gene_end",np.empty(mut_df_1.shape[0]))
mut_df_1.insert(mut_df_1.shape[1],"strand",np.empty(mut_df_1.shape[0]))
mut_df_1.insert(mut_df_1.shape[1],"frame",np.empty(mut_df_1.shape[0]))

chromosome_dfs = {}
for chrom in set(exon_df['chromosome']):
  chrom_mask = exon_df['chromosome'] == chrom
  chromosome_dfs[chrom] = pd.DataFrame(exon_df[chrom_mask])

genome_str = ""
with open(genome_fasta_file, 'r') as file:
  for line in file:
    if '>' in line:
      genome_str = genome_str + "\n" + line
    else:
      genome_str = genome_str + line[:-1]
genome_str = genome_str[1:] # remove first newline

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [ ]:
#@title Function definitions

# Store which genes the mutations are in if they are in exons
def binary_mutation_search(mut_df, chrom_df_arr):
  hits = 0
  df = mut_df.copy()
  for i in df.index:
    chrom_df = chrom_df_arr[mut_df_1.loc[i,'CHROM']]
    chrom_df.sort_values(by='start_pos',axis=0,inplace=True)
    mut_pos = int(df.loc[i,'POS1'])

    lower = -1
    upper = len(chrom_df.index)
    idx = int(upper/2)
    chrom_idx = chrom_df.index

    # Binary search
    while True:
      idx_old = idx
      gene_start = int(chrom_df.loc[chrom_idx[idx],'start_pos'])
      gene_end = int(chrom_df.loc[chrom_idx[idx],'end_pos'])

      if mut_pos < gene_start:
        # Mutation precedes gene, look in previous half
        upper = idx
        idx -= int((idx - lower) / 2)

      elif mut_pos <= gene_end:
        # Mutation within gene [start, end], store id
        df.loc[i,'gene_id'] = chrom_df.loc[chrom_idx[idx],"id"]
        df.loc[i,'gene_start'] = chrom_df.loc[chrom_idx[idx],"start_pos"]
        df.loc[i,'gene_end'] = chrom_df.loc[chrom_idx[idx],"end_pos"]
        df.loc[i,'strand'] = chrom_df.loc[chrom_idx[idx],"strand"]
        df.loc[i,'frame'] = chrom_df.loc[chrom_idx[idx],"frame"]

        hits += 1
        break

      else:
        # Mutation follows gene, look in next half
        lower = idx
        idx += int((upper - idx) / 2)

      if idx == idx_old: break # Mutation not within any gene

  print(f"Number of hits:\n{hits}\n")
  return df


# Extract gene IDs and names with mutations
def get_gene_dict(mut_id_df, gene_df):
  gene_dict = {}
  for gene in mut_id_df['gene_id']:
    match = re.search(r'^ID=[A-Z0-9\.]*',str(gene))
    if match: 
      gene_dict[match[0]] = None

  name_m = gene_df['id'].str.contains("Name=")
  name_df = gene_df[name_m]
  for name in name_df['id']:
    id_match = re.search(r'^ID=[A-Z0-9\.]*',str(name))
    name_match = re.search(r'(Name=)(.*)',str(name))
    if id_match[0] in gene_dict.keys():
      gene_dict[id_match[0]] = name_match[2]
  return gene_dict


# Apply all mutations in the mutation df to a specific gene id
def insert_mutation(mutation_df, gene_id):

  # Filter df by specific gene_id
  mask = mutation_df['gene_id'].str.contains(gene_id)
  df = mutation_df[mask]

  # Define key variables
  first_idx = df.index[0]
  chrom = str(df.loc[first_idx,'CHROM']) + "\n"
  gene_start = int(df.loc[first_idx,'gene_start'])
  gene_end = int(df.loc[first_idx,'gene_end'])
  frame = int(df.loc[first_idx,'frame'])
  chrom_start = genome_str.index(chrom) + len(chrom) - 1 # -1 because genome positions index starting with 1

  # Find gene nt sequence in genome string
  gene = genome_str[chrom_start + gene_start + frame: chrom_start + gene_end + 1]
  old_seq = Seq(gene)

  # Apply each mutation to gene nt sequence
  for i in df.index:
    mut_pos = int(df.loc[i,'POS1'])
    mut_gene_pos = mut_pos - gene_start
    
    gene = gene[:mut_gene_pos] + str(df.loc[i,'ALT1']) + \
      gene[mut_gene_pos + len(str(df.loc[i,'REF1'])):]
  
  # Return new gene sequence (accounting for reverse strand)
  new_seq = Seq(gene)
  if str(df.loc[first_idx,'strand']) == '+':
    return old_seq, new_seq, "+", frame
  else:
    return old_seq.reverse_complement(), new_seq.reverse_complement(), "-", frame

In [ ]:
#@title Print names of genes with mutations

mut_id_df = binary_mutation_search(mut_df_1, chromosome_dfs)

# Remove non genes and print systematic gene names
gene_dict = get_gene_dict(mut_id_df, genome_df)
print("Systematic gene names:\n",np.array(list(gene_dict.keys())))
print("\nNumber of genes with mutations:\n",len(gene_dict.keys()))
print("\nKnown gene names (if in genome dataframe):\n",np.array(list(set(gene_dict.values()))))

In [ ]:
#@title Display mutation dataframe

#Cut mutation dataframe to only mutations within genes
mut_hit_df = mut_id_df.copy()
mut_hit_df.drop('POS2',axis=1,inplace=True)
mut_hit_df.drop('REF2',axis=1,inplace=True)
mut_hit_df.drop('IN_FILE',axis=1,inplace=True)
mut_hit_df.drop('ALT2',axis=1,inplace=True)
for row in mut_hit_df.index:
  if not "ID" in str(mut_hit_df.loc[row,"gene_id"]):
    mut_hit_df.drop(row,inplace=True)

if download_mutation_csv:
  mut_hit_df.to_csv(job_name+".csv")
mut_hit_df

In [ ]:
#@title Apply mutations to peptide sequences and write results to file

def write_mutation_result(outfile, oldpep, newpep):
  if oldpep == newpep:
    outfile.write("Mutation did not cause an amino acid change\n")
    outfile.write(f"{newpep}")
  else:
    outfile.write(f"New peptide sequence:\n{newpep}\n")
  
  # diff_count adapted from https://stackoverflow.com/questions/28423448/counting-differences-between-two-strings
  diff_count = sum(1 for a, b in zip(str(newpep), str(oldpep)) if a != b) + abs(len(new_pep) - len(old_pep))

  if diff_count < 20:
    for i in range(np.min([len(oldpep),len(newpep)])):
      if oldpep[i] != newpep[i]:
        outfile.write(f"AA#{i+1} {oldpep[i]} -> {newpep[i]}, ")
    outfile.write("\n")
    if len(oldpep) != len(newpep):
      outfile.write(f"Old and new peptide lengths do not match\n")
  else:
    for i in range(np.min([len(oldpep),len(newpep)])):
      if oldpep[i] != newpep[i]:
        first_diff = i+1
        break
    outfile.write(f"\nMore than 20 amino acids differ, mutation likely caused frameshift starting at AA#{first_diff}. Individual amino acid changes omitted for brevity.\n")


outfile = open(job_name+".txt", "w")
outfile.write(f"Systematic gene names:\n{np.array(list(gene_dict.keys()))}\n\n")
for gene in set(mut_hit_df['gene_id']):
  old, new, strand, frame = insert_mutation(mut_hit_df,gene)
  outfile.write("\n--------------------------------------------------------------------------------\n\n")
  outfile.write(f"Results for {gene} on {strand} in frame {frame}:\n\n")

  old_pep = old.transcribe().translate()
  new_pep = new.transcribe().translate()
  unshifted_stop_count = sum(1 for a in str(new_pep) if a=='*')

  write_mutation_result(outfile, old_pep, new_pep)
  
  if not ("exon:1" in gene):

    shift1 = len(old) % 3
    shift2 = 3 - shift1
    outfile.write(f"\nExon not at start of protein, recording results in other frames as well:\n")
    
    outfile.write(f"\nFrame {(frame+1)%3}:\n")
    old_shift1 = old[1:]
    new_shift1 = new[1:]
    old_pep = old_shift1.transcribe().translate()
    new_pep = new_shift1.transcribe().translate()
    write_mutation_result(outfile, old_pep, new_pep)

    outfile.write(f"\nFrame {(frame+2)%3}:\n")
    old_shift2 = old[2:]
    new_shift2 = new[2:]
    old_pep = old_shift2.transcribe().translate()
    new_pep = new_shift2.transcribe().translate()
    write_mutation_result(outfile, old_pep, new_pep)
    

outfile.close()
files.download(job_name+".txt")
if download_mutation_csv:
  files.download(job_name+".csv")